In [87]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_palette('husl')

In [88]:
df = pd.read_pickle('data/processed.pickle')
df.head()

,key,title,authors,abstract,year_published,tags,category,skills,skill_cat,types,lang_family
0,rayyan-354359269,An experience of automated assessment in a lar...,"Zampirolli F.A., Borovina Josko J.M., Venero M...",The 2020 pandemic imposed new demands on teach...,2021,"[data_available:False, technique:pre_defined_q...",[dynamic],[correctness],[correctness_dynamic],"[description, evaluation]",[OOP]
1,rayyan-354359270,Semiautomatic generation and assessment of Jav...,"Insa D., Pérez S., Silva J., Tamarit S.",Continuous assessment is essential in educatio...,2021,"[skill:correctness, tool:javassess, interactio...","[dynamic, static]",[correctness],"[correctness_dynamic, correctness_static]","[description, evaluation]",[OOP]
2,rayyan-354359271,A tool for evaluating computer programs from s...,"Vaneck Q., Colart T., Frénay B., Vanderose B.",Computer science studies are more and more pop...,2021,"[data_available:False, tool:pylint, language:p...",[static],"[readability, maintainability]","[readability_static, maintainability_static]",[description],[OOP]
3,rayyan-354359272,Virtual Teaching Assistant for Grading Program...,"Chou C.-Y., Chen Y.-J.",This study proposes an automated programming a...,2021,"[data_available:False, skill:correctness, inte...","[dynamic, static]",[correctness],"[correctness_dynamic, correctness_static]","[description, evaluation]",[Unknown]
4,rayyan-354359273,Analysis of an automatic grading system within...,"Hegarty-Kelly E., Mooney D.A.",Reliable and pedagogically sound automated fee...,2021,"[data_available:False, skill:correctness, inte...","[dynamic, static]",[correctness],"[correctness_dynamic, correctness_static]","[description, evaluation]",[OOP]


# Discussion Stats
## Grading
### Correctness

In [89]:
correct_df = df.copy(deep=True)
correct_df.skills = correct_df.skills.astype(str)
correct_df = correct_df[correct_df.skills.str.contains("^\['correctness'\]$")]
correct_df['tech'] = correct_df.tags.apply(lambda labels: [label for label in labels if label.startswith('technique')])
correct_df[['key', 'title', 'category', 'tech']]

,key,title,category,tech
0,rayyan-354359269,An experience of automated assessment in a lar...,[dynamic],"[technique:pre_defined_questions, technique:un..."
1,rayyan-354359270,Semiautomatic generation and assessment of Jav...,"[dynamic, static]","[technique:code_repair_for_feedback, technique..."
3,rayyan-354359272,Virtual Teaching Assistant for Grading Program...,"[dynamic, static]","[technique:output_matching, technique:unit_tes..."
4,rayyan-354359273,Analysis of an automatic grading system within...,"[dynamic, static]","[technique:unit_testing, technique:pattern_mat..."
5,rayyan-354359274,Evaluating Control-Flow Graph Similarity for G...,[static],"[technique:model_solution_req, technique:patte..."
...,...,...,...,...
114,rayyan-354359408,ArTEMiS - An automatic assessment management s...,"[dynamic, static]","[technique:unit_testing, technique:static_anal..."
115,rayyan-354359409,Linkage objects for generalized instruction in...,"[static, ml]","[technique:machine_learning, technique:pattern..."
116,rayyan-354359410,Automated data-driven hints for computer progr...,"[dynamic, static]","[technique:unit_testing, technique:static_anal..."
117,rayyan-354359411,Learning and teaching numerical methods with a...,"[dynamic, static]","[technique:unit_testing, technique:matching_ru..."


In [90]:
(correct_df.shape[0] / df.shape[0]) * 100

66.11570247933885

In [91]:
correct_df.category.value_counts()

[dynamic]            36
[dynamic, static]    21
[static]             17
[static, ml]          3
[ml]                  2
[dynamic, ml]         1
Name: category, dtype: int64

In [92]:
(correct_df.category.value_counts() / correct_df.shape[0]) * 100

[dynamic]            45.00
[dynamic, static]    26.25
[static]             21.25
[static, ml]          3.75
[ml]                  2.50
[dynamic, ml]         1.25
Name: category, dtype: float64

In [93]:
correct_df['only_dynamic'] = df.category.apply(lambda x: 'dynamic' in str(x) and 'static' not in str(x) and 'ml' not in str(x))
(correct_df[correct_df.only_dynamic].explode('tech').tech.value_counts() / correct_df[correct_df.only_dynamic].shape[0]) * 100

technique:unit_testing                100.000000
technique:ci_cd                        11.111111
technique:model_solution_req           11.111111
technique:trace_examination             5.555556
technique:property_based_testing        5.555556
technique:output_matching               5.555556
technique:pre_defined_questions         2.777778
technique:unit_test_expansion           2.777778
technique:mutation_testing              2.777778
technique:test_case_generation          2.777778
technique:novel                         2.777778
technique:gui_testing                   2.777778
technique:program_synthesis             2.777778
technique:code_repair_for_feedback      2.777778
Name: tech, dtype: float64

In [94]:
correct_df['only_static'] = df.category.apply(lambda x: 'dynamic' not in str(x) and 'static' in str(x) and 'ml' not in str(x))
(correct_df[correct_df.only_static].explode('tech').tech.value_counts() / correct_df[correct_df.only_static].shape[0]) * 100

technique:pattern_matching            70.588235
technique:model_solution_req          41.176471
technique:model_solution_closeness    35.294118
technique:program_repair              35.294118
technique:code_repair_for_feedback    29.411765
technique:clustering                  17.647059
technique:matching_rules              17.647059
technique:trace_examination           17.647059
technique:output_matching             11.764706
technique:graph_analysis              11.764706
technique:control_flow_graph           5.882353
technique:data_driven                  5.882353
technique:token_pattern_matching       5.882353
technique:knowledge_base               5.882353
technique:dynamic_analysis             5.882353
Name: tech, dtype: float64

In [95]:
correct_df['dynamic_static'] = df.category.apply(lambda x: 'dynamic' in str(x) and 'static' in str(x) and 'ml' not in str(x))
(correct_df[correct_df.dynamic_static].explode('tech').tech.value_counts() / correct_df[correct_df.dynamic_static].shape[0]) * 100

technique:unit_testing                100.000000
technique:matching_rules               33.333333
technique:pattern_matching             28.571429
technique:static_analysis              23.809524
technique:style_check                  19.047619
technique:model_solution_req           19.047619
technique:program_repair               14.285714
technique:output_matching               9.523810
technique:dynamic_analysis              9.523810
technique:model_solution_closeness      9.523810
technique:ci_cd                         9.523810
technique:test_case_generation          4.761905
technique:code_repair_for_feedback      4.761905
technique:language_stuctures            4.761905
technique:program_verification          4.761905
technique:fault_localisation            4.761905
technique:knowledge_base                4.761905
Name: tech, dtype: float64

In [109]:
correct_df['ml'] = df.category.apply(lambda x: 'ml' in str(x))
ml_df = correct_df.loc[correct_df.ml]
ml_df[['key', 'title', 'category']]

,key,title,category
13,rayyan-354359285,An Intelligent Online Judge System for Program...,"[dynamic, ml]"
19,rayyan-354359295,Convolutional neural network applied to code a...,[ml]
34,rayyan-354359310,Automatic assessment of programming assignment...,[ml]
61,rayyan-354359341,Source-Code Similarity Measurement: Syntax Tre...,"[static, ml]"
83,rayyan-354359367,Targeted Example Generation for Compilation Er...,"[static, ml]"
115,rayyan-354359409,Linkage objects for generalized instruction in...,"[static, ml]"


### Maintainability

In [97]:
maint_df = df.copy(deep=True)
maint_df.skills = maint_df.skills.astype(str)
maint_df = maint_df[maint_df.skills.str.contains("^\['maintainability'\]$")].copy()
maint_df

,key,title,authors,abstract,year_published,tags,category,skills,skill_cat,types,lang_family
92,rayyan-354359378,A Comparison of Inquiry-Based Conceptual Feedb...,"Cordova L., Carver J., Gershmel N., Walia G.",The feedback provided by current testing educa...,2021,"[interaction:multiple, evaluation:student_surv...",[static],['maintainability'],[maintainability_static],[evaluation],[OOP]


In [98]:
maint_df['tech'] = maint_df.tags.apply(lambda labels: [label for label in labels if label.startswith('technique')])
(maint_df.explode('tech').tech.value_counts() / maint_df.shape[0]) * 100

technique:static_analysis       100.0
technique:model_solution_req    100.0
Name: tech, dtype: float64

### Readability

In [99]:
# Analysing hint based problem solving strategy among novice programmers through gamification technique - Might be removed
read_df = df.copy(deep=True)
read_df.skills = read_df.skills.astype(str)
read_df = read_df[read_df.skills.str.contains("^\['readability'\]$")]
read_df[['key', 'title', 'category']]

,key,title,category
30,rayyan-354359306,The impact of automated code quality feedback ...,[static]
77,rayyan-354359360,Promoting Code Quality via Automated Feedback ...,[static]
79,rayyan-354359362,Measure Students’ Contribution in Web Programm...,[static]
107,rayyan-354359398,Static analyses in python programming courses,[static]


In [100]:
(read_df.shape[0] / df.shape[0]) * 100

3.3057851239669422

In [101]:
read_df.category.value_counts()

[static]    4
Name: category, dtype: int64

### Correctness, Documentation

In [102]:
doc_df = df.copy(deep=True)
doc_df.skills = doc_df.skills.astype(str)
doc_df[doc_df.skills.str.contains("^\['correctness', 'documentation'\]$")]

,key,title,authors,abstract,year_published,tags,category,skills,skill_cat,types,lang_family
29,rayyan-354359305,Developing applications to automatically grade...,Gerdes J.,There are many unique challenges associated wi...,2017,"[data_available:False, skill:correctness, tech...",[static],"['correctness', 'documentation']","[correctness_static, documentation_static]","[description, evaluation]",[OOP]


## Degree of Automation

In [103]:
doa_df = df.copy()
doa_df.tags = doa_df.tags.apply(lambda labels: [str(label) for label in labels if label.startswith('approach')])
doa_df = doa_df.explode('tags')
doa_df.tags.value_counts()

approach:fully_automated    99
approach:semi_automatic     17
approach:unclear             5
Name: tags, dtype: int64

In [104]:
(doa_df.tags.value_counts() /  doa_df.shape[0]) * 100

approach:fully_automated    81.818182
approach:semi_automatic     14.049587
approach:unclear             4.132231
Name: tags, dtype: float64

In [105]:
doa_df[doa_df.tags.str.contains('semi')]

,key,title,authors,abstract,year_published,tags,category,skills,skill_cat,types,lang_family
1,rayyan-354359270,Semiautomatic generation and assessment of Jav...,"Insa D., Pérez S., Silva J., Tamarit S.",Continuous assessment is essential in educatio...,2021,approach:semi_automatic,"[dynamic, static]",[correctness],"[correctness_dynamic, correctness_static]","[description, evaluation]",[OOP]
3,rayyan-354359272,Virtual Teaching Assistant for Grading Program...,"Chou C.-Y., Chen Y.-J.",This study proposes an automated programming a...,2021,approach:semi_automatic,"[dynamic, static]",[correctness],"[correctness_dynamic, correctness_static]","[description, evaluation]",[Unknown]
10,rayyan-354359280,Syntactic and Semantic Analysis for Extended F...,"Andujar C., Vijulie C.R., Vinacua A., Santos B...",Modern computer graphics courses require stude...,2020,approach:semi_automatic,"[dynamic, static]",[correctness],"[correctness_dynamic, correctness_static]",[description],[Graphics]
14,rayyan-354359287,Teaching programming at scale,"Kaplan A., Keim J., Schneider Y.R., Walter M.,...",Teaching programming is a difficult task and t...,2020,approach:semi_automatic,"[dynamic, static]","[correctness, readability]","[readability_static, correctness_dynamic]",[experience],[OOP]
16,rayyan-354359291,A parser-based tool to assist instructors in g...,"Andujar C., Vijulie C.R., Vinacua A.",Although online e-learning environments are in...,2019,approach:semi_automatic,"[dynamic, static]",[correctness],"[correctness_dynamic, correctness_static]","[description, evaluation]","[OOP, Graphics]"
19,rayyan-354359295,Convolutional neural network applied to code a...,"De Souza F.R., De Assis Zampirolli F., Kobayas...",Thousands of students have their assignments e...,2019,approach:semi_automatic,[ml],[correctness],[correctness_ml],"[description, evaluation]",[OOP]
36,rayyan-354359313,Enhancing the Learning of Database Access Prog...,B. Pérez,Database access programming is a noteworthy co...,2021,approach:semi_automatic,[dynamic],[correctness],[correctness_dynamic],"[description, evaluation]",[OOP]
37,rayyan-354359314,MAESTRO: a semi-autoMAted Evaluation SysTem fo...,A. Bertagnon and M. Gavanelli,Many works in the literature highlight the imp...,2020,approach:semi_automatic,[dynamic],[correctness],[correctness_dynamic],"[description, evaluation]","[OOP, Functional]"
41,rayyan-354359318,Gradeer: An Open-Source Modular Hybrid Grader,B. Clegg and M. -C. Villa-Uriol and P. McMinn ...,Automated assessment has been shown to greatly...,2021,approach:semi_automatic,"[dynamic, static]","[correctness, readability]",[correctness_dynamic],"[description, evaluation]",[OOP]
44,rayyan-354359321,Automating Testing of Visual Observed Concurrency,P. Dewan and A. Wortas and Z. Liu and S. Georg...,Existing techniques for automating the testing...,2021,approach:semi_automatic,[dynamic],[correctness],[correctness_dynamic],"[description, evaluation]",[OOP]


## Language Paradigms

In [106]:
lp_df = df.copy()
lp_df.lang_family.value_counts()

[OOP]                83
[Unknown]             9
[Functional]          8
[Agnostic]            5
[Low-Level]           3
[Graphics]            2
[OOP, Graphics]       2
[OOP, Functional]     2
[OOP, Agnostic]       2
[Web]                 2
[Query]               1
[OOP, Web]            1
[Logic]               1
Name: lang_family, dtype: int64

In [107]:
(lp_df.lang_family.value_counts() / lp_df.shape[0]) * 100

[OOP]                68.595041
[Unknown]             7.438017
[Functional]          6.611570
[Agnostic]            4.132231
[Low-Level]           2.479339
[Graphics]            1.652893
[OOP, Graphics]       1.652893
[OOP, Functional]     1.652893
[OOP, Agnostic]       1.652893
[Web]                 1.652893
[Query]               0.826446
[OOP, Web]            0.826446
[Logic]               0.826446
Name: lang_family, dtype: float64

## Evaluation

In [108]:
eval_df = df.copy()
eval_df.tags = eval_df.tags.apply(lambda labels: [str(label) for label in labels if label.startswith('evaluation')])
eval_df = eval_df.explode('tags')
eval_df.tags.value_counts()

evaluation:student_survey             38
evaluation:manual_grading             36
evaluation:analytics                  20
evaluation:auto_grading               15
evaluation:none                       11
evaluation:user_study                  6
evaluation:grading_accuracy            6
evaluation:teacher_survey              6
evaluation:compared_to_other_tools     5
evaluation:anecdotal                   5
evaluation:usage                       4
evaluation:engagement                  3
evaluation:grade_correlation           2
evaluation:feedback                    2
evaluation:correlation_grades          2
evaluation:ml_models                   1
Name: tags, dtype: int64